# Exercício

Considere a base de dados [Spambase](https://archive.ics.uci.edu/ml/datasets/Spambase). Identifique, usando validação cruzada de 5 pastas, qual classificador apresenta melhores resultados:

    Árvore de Decisão com max_depth= None ou 2
    KNN com número de vizinhos 3 ou 5 e ponderação uniforme ou pela distância



In [41]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


In [21]:
dados = pd.read_csv("spambase.data", header=None)

In [22]:
X, y = dados.drop(57, axis=1), dados[57]

In [23]:
# Algoritmos
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier()

In [28]:
param_grid_tree = [{'max_depth': [None, 2]}]
param_grid_knn = [{'n_neighbors': [3, 5], 'weights': ['uniform','distance']}]

grid_search_knn = GridSearchCV(knn, param_grid_knn, cv = 5)
grid_search_tree = GridSearchCV(tree, param_grid_tree, cv = 5)

In [29]:
grid_search_knn.fit(X,y)
grid_search_tree.fit(X,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [None, 2]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [30]:
means_knn = grid_search_knn.cv_results_['mean_test_score']
stds_knn = grid_search_knn.cv_results_['std_test_score']

means_tree = grid_search_tree.cv_results_['mean_test_score']
stds_tree = grid_search_tree.cv_results_['std_test_score']

In [38]:
print(f"KNN:  {np.mean(means_knn):.3f} +/- {np.mean(stds_knn):.3f}\n"+
      f"TREE: {np.mean(means_tree):.3f} +/- {np.mean(stds_tree):.3f}")

KNN:  0.782 +/- 0.042
TREE: 0.862 +/- 0.058


## OUTRA SOLUCAO

In [45]:

param_grid= [{'max_depth': [None, 2]} ,  # Tree
             {'n_neighbors': [3, 5], 'weights': ['uniform','distance']}] # KNN

# Deve ser criado fora do loop, para sempre comparar a mesma coisa
divisor = StratifiedKFold(n_splits = 5, random_state = 1) 
for clf, parameters in zip([tree, knn], param_grid):
    grid_search = GridSearchCV(clf, parameters, cv = divisor)
    grid_search.fit(X, y)
    
    
    print( grid_search.best_estimator_)
    print( np.mean(grid_search.cv_results_['mean_test_score']))
    print( np.mean(grid_search.cv_results_['std_test_score']))
    print("-"*20)
    

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.8619865246685503
0.058244162307406255
--------------------
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance')
0.7815148880678113
0.042245469237265076
--------------------
